<a href="https://colab.research.google.com/github/CMWENLIU/Drug_label_embedding/blob/master/drug_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip3 install --quiet wget
!pip install html2text

In [61]:
import os
import csv
import pandas as pd
import re
import wget
import html2text
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
import nltk.data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
urls = ["https://raw.githubusercontent.com/CMWENLIU/Drug_label_embedding/master/fdalabel-query-1132.csv",
       "https://raw.githubusercontent.com/CMWENLIU/Drug_label_embedding/master/indications_humanrx.xlsx",
       ]
[wget.download(u) for u in urls]
 
    
  

['fdalabel-query-1132 (1).csv', 'indications_humanrx (1).xlsx']

In [0]:
def clean_str(string):
   
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = string.replace("\\","")
    return string.strip().lower()

In [0]:
df = pd.read_csv('fdalabel-query-1132.csv')
df1 = df[['Most Recent SPL Date (YYYY/MM/DD)','Active Ingredient UNII(s)', 'SET ID']]
df2 = pd.read_excel('indications_humanrx.xlsx')

In [0]:
#print(df1.shape)
#print(df2.shape)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
df1 = df1.groupby('Active Ingredient UNII(s)').first()
df1.to_csv('res.csv')
res_df = df1.join(df2.set_index('SET_ID'), on='SET ID')

  

In [80]:
#texts = []
h = html2text.HTML2Text()
h.ignore_links = True
#for v in res_df['CONTENT_XML'].tolist():
#  print(h.handle(v))
#res_df['CONTENT_XML'] = texts
#res_df.to_csv('newres.csv')

set_ids, xmls, texts, sentences,  = [], [], [], []
for idx, row in res_df.iterrows():
  v = str(res_df.loc[idx, 'CONTENT_XML'])
  if v:
    text = h.handle(v).replace("``", "")
    subs = sent_tokenize(text)
    for t in subs:
      if len(t) > 30:
        set_ids.append(res_df.loc[idx, 'SET ID'])
        xmls.append(res_df.loc[idx, 'CONTENT_XML'])
        texts.append(text)
        sentences.append(clean_str(t))
final_df = pd.DataFrame()
final_df['set_ids'] = set_ids
#final_df['xmls'] = xmls
#final_df['texts'] = texts
final_df['sentences'] = sentences
final_df.to_csv('final.csv') 
final_df.head()
    




,set_ids,sentences
0,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,indications and usage minocycline hydrochlorid...
1,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,respiratory tract infections caused by mycopla...
2,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,lymphogranuloma venereum caused by chlamydia t...
3,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,psittacosis ( ornithosis ) due to chlamydia ps...
4,ad8678de-ee78-d061-ea5f-d2a6b949b0e5,"trachoma caused by chlamydia trachomatis , alt..."


In [0]:
com_df = df1.set_index('SET ID').join(df2.set_index('SET_ID'))

In [19]:
print(com_df.shape)
com_df.head()

(40580, 3)


,Most Recent SPL Date (YYYY/MM/DD),Active Ingredient UNII(s),CONTENT_XML
00011703-bc55-4c0c-858c-149dc674bc3c,20171230,A4P49JAZ9H,"<section xmlns=""urn:hl7-org:v3"" ID=""s9"" xmlns:..."
000155a8-709c-44e5-a75f-cd890f3a7caf,20190124,57Y76R9ATQ,"<section xmlns=""urn:hl7-org:v3"" ID=""ID_2f9d5bc..."
0002ad27-779d-42ab-83b5-bc65453412a1,20181218,BSB9VJ5TUT,"<section xmlns=""urn:hl7-org:v3"" xmlns:xsi=""htt..."
0002eded-a283-4590-a78c-b6ff2027e684,20160809,6CW7F3G59X,"<section xmlns=""urn:hl7-org:v3"" ID=""LINK_27310..."
0003458f-352a-46fa-9d99-230daa76ae29,20180223,6KY687524K,"<section xmlns=""urn:hl7-org:v3"" ID=""INDICATION..."
